Cet algorithme vise à estimer le prix d'une option Bermudéenne via différents estimateurs

Les estimateurs : high, low 1 et low 2, avec l'implémentation naïveLes estimateurs : high, low 1 et low 2, avec l'implémentation naïve

In [4]:
import numpy as np
import scipy.stats as sps
from time import time
import time

r, sigma, X_0, K = 0.1, 0.2, 100, 100

m, b, n = 3, 10, 50
delta_t = 3/m      #maturité = 3 ans; m est le nombre d'occasions d'exercice
isize=b//2        #taille du tas de comparaison pour le premier low estimator. 

tic1, toc1, toc2, toc3, toc4 = 0, 0, 0, 0, 0   #calcul du temps écoulé.

Sample = np.zeros(n)
SampleLow = np.zeros(n)
SampleLow2 = np.zeros(n)
for k in range(n):
    tic1 = tic1 + time.perf_counter()
    
    X = np.zeros((b**m,m+1))   #X est l'arbre
    X[0,0] = X_0
    for i in range(m):
        for j in range(b**(i+1)):
            X[j,i+1]=X[j//b,i]*np.exp((r-((sigma)**2)/2)*delta_t + sigma*np.sqrt(delta_t)*np.random.randn())

    toc1 = toc1 + time.perf_counter()
        
    V = np.zeros((b**m,m+1))   #V est le high estimator
    V[:,m]=np.maximum(K-X[:,m],0)
    for i in range(m):
        for j in range(b**(m-1-i)):
            V[j,m-1-i]=np.maximum(np.maximum(K-X[j,m-1-i],0),np.exp(-r)*np.mean(V[j*b:(j+1)*b,m-i]))
    Sample[k] = V[0,0]
    
    toc2 = toc2 + time.perf_counter()
    
    Vv = np.zeros((b**m,m+1))   #Vv est le premier low estimator
    Vv[:,m]=np.maximum(K-X[:,m],0)
    for i in range(m):
        for j in range(b**(m-1-i)):
            Vv1=np.exp(-r)*np.mean(Vv[j*b:j*b+isize,m-i])
            Vv2=np.exp(-r)*np.mean(Vv[j*b+isize:j*b+b,m-i])
            a=np.maximum(K-X[j,m-1-i],0)
            if Vv1<=a:
                Vv[j,m-1-i]=a
            else:
                Vv[j,m-1-i]=Vv2
    SampleLow[k] = Vv[0,0]
    
    toc3 = toc3 + time.perf_counter()
    
    VLow = np.zeros((b**m,m+1))   #VLow est le second low estimator
    VLow[:,m]=np.maximum(K-X[:,m],0)
    for i in range(m):
        for j in range(b**(m-1-i)):
            Vtemp=np.zeros(b)
            for h in range(b):
                hlist=[j*b + ii for ii in range(b) if ii not in [h]]
                VLow1=np.exp(-r)*np.mean(VLow[hlist,m-i])
                VLow2=np.exp(-r)*VLow[j*b +h,m-i]
                a=np.maximum(K-X[j,m-1-i],0)
                if VLow1<=a:
                    Vtemp[h]=a
                else:
                    Vtemp[h]=VLow2
            VLow[j,m-1-i]=np.mean(Vtemp)
    SampleLow2[k] = VLow[0,0]
    
    toc4 = toc4 + time.perf_counter()

print("Implémentation naïve")
print("Time =",toc4-tic1," Time for X =",toc1-tic1)
print("Time for the High estimator =",toc2-toc1)
print("Time for the first low estimator =",toc3-toc2)
print("Time for the second low estimator =",toc4-toc3)
Estimateur=np.mean(Sample)
Ecart=np.std(Sample)
print("High Vo(n,b) =",Estimateur," Error =",100*2*1.96*Ecart/(np.sqrt(n)*Estimateur),"%")
print("High Confidence interval at 95% =",[Estimateur-1.96*Ecart/np.sqrt(n),Estimateur+1.96*Ecart/np.sqrt(n)])
EstimateurLow=np.mean(SampleLow)
EcartLow=np.std(SampleLow)
print("Low Vo(n,b) =",EstimateurLow," Error =",100*2*1.96*EcartLow/(np.sqrt(n)*EstimateurLow),"%")
print("Low Confidence interval at 95% =",[EstimateurLow-1.96*EcartLow/np.sqrt(n),EstimateurLow+1.96*EcartLow/np.sqrt(n)])
EstimateurLow2=np.mean(SampleLow2)
EcartLow2=np.std(SampleLow2)
print("Low2 Vo(n,b) =",EstimateurLow2," Error =",100*2*1.96*EcartLow2/(np.sqrt(n)*EstimateurLow2),"%")
print("Low2 Confidence interval at 95% =",[EstimateurLow2-1.96*EcartLow2/np.sqrt(n),EstimateurLow2+1.96*EcartLow2/np.sqrt(n)])
print("Final Confidence interval at 95% =",[EstimateurLow2-1.96*EcartLow2/np.sqrt(n),Estimateur+1.96*Ecart/np.sqrt(n)])
FE=(EstimateurLow2-1.96*EcartLow2/np.sqrt(n) + Estimateur+1.96*Ecart/np.sqrt(n))/2
EE=Estimateur+1.96*Ecart/np.sqrt(n) - (EstimateurLow2-1.96*EcartLow2/np.sqrt(n))
print("Final estimator =",FE,"Final error =",100*EE/FE,"%")

Implémentation naïve
Time = 1.2287407999992865  Time for X = 0.17668679999951564
Time for the High estimator = 0.05909909999900265
Time for the first low estimator = 0.08521419999851787
Time for the second low estimator = 0.9077407000022504
High Vo(n,b) = 4.688466556458289  Error = 28.217412435636465 %
High Confidence interval at 95% = [4.0269845838869305, 5.349948529029648]
Low Vo(n,b) = 4.267803599039797  Error = 41.883846699115026 %
Low Confidence interval at 95% = [3.374043440619226, 5.161563757460368]
Low2 Vo(n,b) = 4.3209310541336325  Error = 30.474898787387072 %
Low2 Confidence interval at 95% = [3.662531371423632, 4.979330736843633]
Final Confidence interval at 95% = [3.662531371423632, 5.349948529029648]
Final estimator = 4.5062399502266395 Final error = 37.446234027576544 %


Implémentation récursive

In [5]:
import numpy as np
import scipy.stats as sps
from time import time
import time

r, sigma, X_0, K = 0.1, 0.2, 100, 100

m, b, n = 3, 10, 50
delta_t = 3/m
isize=b//2

tic = time.perf_counter()

def Vrecursif(x,k):
    if k==m:  #au dernier instant, on prend juste le payoff
        return [np.maximum(K-x,0),np.maximum(K-x,0),np.maximum(K-x,0)]
    else:
        Vi=np.zeros((b,3))
        for i in range(b):
            Vi[i,:]=Vrecursif(x*np.exp((r-(sigma**2)/2)*delta_t + sigma*np.sqrt(delta_t)*np.random.randn()),k+1)
        
        Vv1=np.exp(-r)*np.mean(Vi[0:isize,2])  #calcul pour le premier estimateur bas
        Vv2=np.exp(-r)*np.mean(Vi[isize:b,2])
        a=np.maximum(K-x,0)
        if Vv1<=a:
            Vv=a
        else:
            Vv=Vv2
        
        VTemp=np.zeros(b)  #calcul pour le second estimateur bas
        for h in range(b):
            hlist=[i for i in range(b) if i not in [h]]
            V1=np.exp(-r)*np.mean(Vi[hlist,1])
            V2=np.exp(-r)*Vi[h,1]
            a=np.maximum(K-x,0)
            if V1<=a:
                VTemp[h]=a
            else:
                VTemp[h]=V2
        return([np.maximum(np.maximum(K-x,0),np.exp(-r)*np.mean(Vi[:,0])),np.mean(VTemp),Vv])
                 #l'estimateur haut est calculé directement
Sample = np.zeros((n,3))
for k in range(n):
        Sample[k,:] = Vrecursif(X_0,0)

toc = time.perf_counter()

print("Implémentation récursive")
print("Time=",toc-tic)
Estimateur=np.mean(Sample[:,0])
Ecart=np.std(Sample[:,0])
print("Vo(n,b)=",Estimateur," Error =",100*2*1.96*Ecart/(np.sqrt(n)*Estimateur),"%")
print("Confidence interval at 95%=",[Estimateur-1.96*Ecart/np.sqrt(n),Estimateur+1.96*Ecart/np.sqrt(n)])
EstimateurLow=np.mean(Sample[:,2])
EcartLow=np.std(Sample[:,2])
print("Low Vo(n,b)=",EstimateurLow," Error =",100*2*1.96*EcartLow/(np.sqrt(n)*EstimateurLow),"%")
print("Low Confidence interval at 95%=",[EstimateurLow-1.96*EcartLow/np.sqrt(n),EstimateurLow+1.96*EcartLow/np.sqrt(n)])
EstimateurLow2=np.mean(Sample[:,1])
EcartLow2=np.std(Sample[:,1])
print("Low2 Vo(n,b)=",EstimateurLow2," Error =",100*2*1.96*EcartLow2/(np.sqrt(n)*EstimateurLow2),"%")
print("Low2 Confidence interval at 95%=",[EstimateurLow2-1.96*EcartLow2/np.sqrt(n),EstimateurLow2+1.96*EcartLow2/np.sqrt(n)])
print("Final Confidence interval at 95% =",[EstimateurLow2-1.96*EcartLow2/np.sqrt(n),Estimateur+1.96*Ecart/np.sqrt(n)])
FE=(EstimateurLow2-1.96*EcartLow2/np.sqrt(n) + Estimateur+1.96*Ecart/np.sqrt(n))/2
EE=Estimateur+1.96*Ecart/np.sqrt(n) - (EstimateurLow2-1.96*EcartLow2/np.sqrt(n))
print("Final estimator =",FE,"Final error =",100*EE/FE,"%")

Implémentation récursive
Time= 1.4925720999999896
Vo(n,b)= 5.462020260482247  Error = 21.603121138869852 %
Confidence interval at 95%= [4.87203683373145, 6.052003687233045]
Low Vo(n,b)= 4.940764207002452  Error = 36.663117725495255 %
Low Confidence interval at 95%= [4.035045108126232, 5.846483305878673]
Low2 Vo(n,b)= 5.136407817004494  Error = 23.098727744733132 %
Low2 Confidence interval at 95%= [4.543185388249965, 5.729630245759022]
Final Confidence interval at 95% = [4.543185388249965, 6.052003687233045]
Final estimator = 5.297594537741504 Final error = 28.481196290766462 %


Méthode de pruning

In [6]:
import numpy as np
import scipy.stats as sps
from time import time
import time

r, sigma, X_0, K = 0.1, 0.2, 100, 100

m, b, n = 3, 50, 500
delta_t = 3/m

tic = time.perf_counter()

def Vrecursif(x,k):
    if k==m:
        return [np.maximum(K-x,0),np.maximum(K-x,0)]
    elif np.maximum(K-x,0)==0:  #si le payoff est trop bas, on ne simule qu'une branche 
        return [np.exp(-r)*z for z in Vrecursif(x*np.exp((r-(sigma**2)/2)*delta_t + sigma*np.sqrt(delta_t)*np.random.randn()),k+1)]
    else:
        Vi=np.zeros((b,2))
        for i in range(b):
            Vi[i,:]=Vrecursif(x*np.exp((r-(sigma**2)/2)*delta_t + sigma*np.sqrt(delta_t)*np.random.randn()),k+1)
        VTemp=np.zeros(b)  #calcul pour l'estimateur bas
        for h in range(b):
            hlist=[i for i in range(b) if i not in [h]]
            V1=np.exp(-r)*np.mean(Vi[hlist,1])
            V2=np.exp(-r)*Vi[h,1]
            a=np.maximum(K-x,0)
            if V1<=a:
                VTemp[h]=a
            else:
                VTemp[h]=V2
        return([np.maximum(np.maximum(K-x,0),np.exp(-r)*np.mean(Vi[:,0])),np.mean(VTemp)])
    
Sample = np.zeros((n,2))
for k in range(n):
        Sample[k,:] = Vrecursif(X_0,0)

toc = time.perf_counter()

print("Méthode de pruning")
print("Time=",toc-tic)
Estimateur=np.mean(Sample[:,0])
Ecart=np.std(Sample[:,0])
print("Vo(n,b)=",Estimateur," Error =",100*2*1.96*Ecart/(np.sqrt(n)*Estimateur),"%")
print("Confidence interval at 95%=",[Estimateur-1.96*Ecart/np.sqrt(n),Estimateur+1.96*Ecart/np.sqrt(n)])
EstimateurLow=np.mean(Sample[:,1])
EcartLow=np.std(Sample[:,1])
print("Low Vo(n,b)=",EstimateurLow," Error =",100*2*1.96*EcartLow/(np.sqrt(n)*EstimateurLow),"%")
print("Low Confidence interval at 95%=",[EstimateurLow-1.96*EcartLow/np.sqrt(n),EstimateurLow+1.96*EcartLow/np.sqrt(n)])
print("Final Confidence interval at 95% =",[EstimateurLow-1.96*EcartLow/np.sqrt(n),Estimateur+1.96*Ecart/np.sqrt(n)])
FE=(EstimateurLow-1.96*EcartLow2/np.sqrt(n) + Estimateur+1.96*Ecart/np.sqrt(n))/2
EE=Estimateur+1.96*Ecart/np.sqrt(n) - (EstimateurLow-1.96*EcartLow/np.sqrt(n))
print("Final estimator =",FE,"Final error =",100*EE/FE,"%")

Méthode de pruning
Time= 8.02338659999998
Vo(n,b)= 5.310177210026745  Error = 24.526468543242867 %
Confidence interval at 95%= [4.658977738522914, 5.9613766815305755]
Low Vo(n,b)= 5.228367230710559  Error = 24.911228473832562 %
Low Confidence interval at 95%= [4.577141977563909, 5.879592483857208]
Final Confidence interval at 95% = [4.577141977563909, 5.9613766815305755]
Final estimator = 5.501075254422503 Final error = 25.162984324816005 %


2ème méthode de pruning

In [7]:
import numpy as np
import scipy.stats as sps
from time import time
import time

r, sigma, X_0, K = 0.1, 0.2, 100, 100

m, b, n = 3, 50, 500
delta_t = 3/m

tic = time.perf_counter()

def Vrecursif(x,k):
    if k==m:
        return [np.maximum(K-x,0),np.maximum(K-x,0)]
    else:
        m1=m-k      #calcul de la valeur de l'options européenne expirant en m partant de x.
        d1=(np.log(x/K)+(r+sigma**2/2)*m1)/(sigma*np.sqrt(m1))
        d2=d1-sigma*np.sqrt(m1)
        EspEuro= (-x*sps.norm.cdf(-d1) + K*np.exp(-r*m1)*sps.norm.cdf(-d2)) 
        if np.maximum(K-x,0)<EspEuro:  #si le payoff est trop bas, on ne simule qu'une branche 
            return [np.exp(-r)*z for z in Vrecursif(x*np.exp((r-(sigma**2)/2)*delta_t + sigma*np.sqrt(delta_t)*np.random.randn()),k+1)]
        else:
            Vi=np.zeros((b,2))
            for i in range(b):
                Vi[i,:]=Vrecursif(x*np.exp((r-(sigma**2)/2)*delta_t + sigma*np.sqrt(delta_t)*np.random.randn()),k+1)
            VTemp=np.zeros(b)  #calcul pour l'estimateur bas
            for h in range(b):
                hlist=[i for i in range(b) if i not in [h]]
                V1=np.exp(-r)*np.mean(Vi[hlist,1])
                V2=np.exp(-r)*Vi[h,1]
                a=np.maximum(K-x,0)
                if V1<=a:
                    VTemp[h]=a
                else:
                    VTemp[h]=V2
            return([np.maximum(np.maximum(K-x,0),np.exp(-r)*np.mean(Vi[:,0])),np.mean(VTemp)])
    
Sample = np.zeros((n,2))
for k in range(n):
        Sample[k,:] = Vrecursif(X_0,0)

toc = time.perf_counter()

print("2ème méthode de pruning")
print("Time=",toc-tic)
Estimateur=np.mean(Sample[:,0])
Ecart=np.std(Sample[:,0])
print("Vo(n,b)=",Estimateur," Error =",100*2*1.96*Ecart/(np.sqrt(n)*Estimateur),"%")
print("Confidence interval at 95%=",[Estimateur-1.96*Ecart/np.sqrt(n),Estimateur+1.96*Ecart/np.sqrt(n)])
EstimateurLow=np.mean(Sample[:,1])
EcartLow=np.std(Sample[:,1])
print("Low Vo(n,b)=",EstimateurLow," Error =",100*2*1.96*EcartLow/(np.sqrt(n)*EstimateurLow),"%")
print("Low Confidence interval at 95%=",[EstimateurLow-1.96*EcartLow/np.sqrt(n),EstimateurLow+1.96*EcartLow/np.sqrt(n)])
print("Final Confidence interval at 95% =",[EstimateurLow-1.96*EcartLow/np.sqrt(n),Estimateur+1.96*Ecart/np.sqrt(n)])
FE=(EstimateurLow-1.96*EcartLow2/np.sqrt(n) + Estimateur+1.96*Ecart/np.sqrt(n))/2
EE=Estimateur+1.96*Ecart/np.sqrt(n) - (EstimateurLow-1.96*EcartLow/np.sqrt(n))
print("Final estimator =",FE,"Final error =",100*EE/FE,"%")

2ème méthode de pruning
Time= 6.7021755999999755
Vo(n,b)= 5.012403212189325  Error = 26.49146521012597 %
Confidence interval at 95%= [4.348473685615139, 5.67633273876351]
Low Vo(n,b)= 4.978323650873905  Error = 26.67149733313445 %
Low Confidence interval at 95%= [4.314426920985087, 5.642220380762722]
Final Confidence interval at 95% = [4.314426920985087, 5.67633273876351]
Final estimator = 5.233531493120644 Final error = 26.022692699348738 %


Last step removal

In [8]:
import numpy as np
import scipy.stats as sps
from time import time
import time

r, sigma, X_0, K = 0.1, 0.2, 100, 100

m, b, n = 3, 10, 50
delta_t = 3/m

tic = time.perf_counter()

def Vrecursif(x,k):
    if k==m-1:  #on prend le maximum entre le payoff en m-1 et la valeur de l'options européenne expirant en m
        d1=(np.log(x/K)+(r+sigma**2/2)*1)/(sigma*np.sqrt(1))
        d2=d1-sigma*np.sqrt(1)  #calcul de la valeur de l'options européenne expirant en m partant de x.
        EspVC= (-x*sps.norm.cdf(-d1) + K*np.exp(-r*1)*sps.norm.cdf(-d2))
        return [np.maximum(np.maximum(K-x,0),EspVC),np.maximum(np.maximum(K-x,0),EspVC)]
    else:
        Vi=np.zeros((b,2))
        for i in range(b):
            Vi[i,:]=Vrecursif(x*np.exp((r-(sigma**2)/2)*delta_t + sigma*np.sqrt(delta_t)*np.random.randn()),k+1)
        VTemp=np.zeros(b)  
        for h in range(b):
            hlist=[i for i in range(b) if i not in [h]]
            V1=np.exp(-r)*np.mean(Vi[hlist,1])
            V2=np.exp(-r)*Vi[h,1]
            a=np.maximum(K-x,0)
            if V1<=a:
                VTemp[h]=a
            else:
                VTemp[h]=V2
        return([np.maximum(np.maximum(K-x,0),np.exp(-r)*np.mean(Vi[:,0])),np.mean(VTemp)])
    
Sample = np.zeros((n,2))
for k in range(n):
        Sample[k,:] = Vrecursif(X_0,0)


toc = time.perf_counter()

print("Last step removal")
print("Time=",toc-tic)
Estimateur=np.mean(Sample[:,0])
Ecart=np.std(Sample[:,0])
print("Vo(n,b)=",Estimateur," Error =",100*2*1.96*Ecart/(np.sqrt(n)*Estimateur),"%")
print("Confidence interval at 95%=",[Estimateur-1.96*Ecart/np.sqrt(n),Estimateur+1.96*Ecart/np.sqrt(n)])
EstimateurLow=np.mean(Sample[:,1])
EcartLow=np.std(Sample[:,1])
print("Low Vo(n,b)=",EstimateurLow," Error =",100*2*1.96*EcartLow/(np.sqrt(n)*EstimateurLow),"%")
print("Low Confidence interval at 95%=",[EstimateurLow-1.96*EcartLow/np.sqrt(n),EstimateurLow+1.96*EcartLow/np.sqrt(n)])
print("Final Confidence interval at 95% =",[EstimateurLow-1.96*EcartLow/np.sqrt(n),Estimateur+1.96*Ecart/np.sqrt(n)])
FE=(EstimateurLow-1.96*EcartLow2/np.sqrt(n) + Estimateur+1.96*Ecart/np.sqrt(n))/2
EE=Estimateur+1.96*Ecart/np.sqrt(n) - (EstimateurLow-1.96*EcartLow/np.sqrt(n))
print("Final estimator =",FE,"Final error =",100*EE/FE,"%")

Last step removal
Time= 1.1157254999999964
Vo(n,b)= 4.958805229817479  Error = 23.611942795664856 %
Confidence interval at 95%= [4.373370102711009, 5.5442403569239485]
Low Vo(n,b)= 4.830257494003698  Error = 23.975228267048088 %
Low Confidence interval at 95%= [4.251224863966907, 5.409290124040489]
Final Confidence interval at 95% = [4.251224863966907, 5.5442403569239485]
Final estimator = 4.890637711086558 Final error = 26.438586731254134 %


Variable antithétique

In [9]:
import numpy as np
import scipy.stats as sps
from time import time
import time

r, sigma, X_0, K = 0.1, 0.2, 100, 100

m, b, n = 3, 50, 20
delta_t = 3/m

tic = time.perf_counter()

def Vrecursif(x,y,k):
    if k==m: 
        return [(np.maximum(K-x,0)+np.maximum(K-y,0))/2,(np.maximum(K-x,0)+np.maximum(K-y,0))/2]
    else:
        Vi=np.zeros((b,2))
        for i in range(b):
            Ran = np.random.randn()
            xplus=x*np.exp((r-(sigma**2)/2)*delta_t + sigma*np.sqrt(delta_t)*Ran)
            xmoins=y*np.exp((r-(sigma**2)/2)*delta_t - sigma*np.sqrt(delta_t)*Ran)
            Vi[i,:]=Vrecursif(xplus,xmoins,k+1)
        VTemp=np.zeros(b)  
        for h in range(b):
            hlist=[i for i in range(b) if i not in [h]]
            V1=np.exp(-r)*np.mean(Vi[hlist,1])
            V2=np.exp(-r)*Vi[h,1]
            a=(np.maximum(K-x,0)+np.maximum(K-y,0))/2
            if V1<=a:
                VTemp[h]=a
            else:
                VTemp[h]=V2
        return([np.maximum((np.maximum(K-x,0)+np.maximum(K-y,0))/2,np.exp(-r)*np.mean(Vi[:,0])),np.mean(VTemp)])
    
Sample = np.zeros((n,2))
for k in range(n):
        Sample[k,:] = Vrecursif(X_0,X_0,0)

toc = time.perf_counter()

print("Variable antithétique")
print("Time=",toc-tic)
Estimateur=np.mean(Sample[:,0])
Ecart=np.std(Sample[:,0])
print("Vo(n,b)=",Estimateur," Error =",100*2*1.96*Ecart/(np.sqrt(n)*Estimateur),"%")
print("Confidence interval at 95%=",[Estimateur-1.96*Ecart/np.sqrt(n),Estimateur+1.96*Ecart/np.sqrt(n)])
EstimateurLow=np.mean(Sample[:,1])
EcartLow=np.std(Sample[:,1])
print("Low Vo(n,b)=",EstimateurLow," Error =",100*2*1.96*EcartLow/(np.sqrt(n)*EstimateurLow),"%")
print("Low Confidence interval at 95%=",[EstimateurLow-1.96*EcartLow/np.sqrt(n),EstimateurLow+1.96*EcartLow/np.sqrt(n)])
print("Final Confidence interval at 95% =",[EstimateurLow-1.96*EcartLow/np.sqrt(n),Estimateur+1.96*Ecart/np.sqrt(n)])
FE=(EstimateurLow-1.96*EcartLow2/np.sqrt(n) + Estimateur+1.96*Ecart/np.sqrt(n))/2
EE=Estimateur+1.96*Ecart/np.sqrt(n) - (EstimateurLow-1.96*EcartLow/np.sqrt(n))
print("Final estimator =",FE,"Final error =",100*EE/FE,"%")

Variable antithétique
Time= 107.87394929999999
Vo(n,b)= 4.941930162434581  Error = 8.040389705989663 %
Confidence interval at 95%= [4.743254940405786, 5.140605384463375]
Low Vo(n,b)= 4.907246639517398  Error = 8.248561206695223 %
Low Confidence interval at 95%= [4.704858018205354, 5.1096352608294415]
Final Confidence interval at 95% = [4.704858018205354, 5.140605384463375]
Final estimator = 4.554942503500067 Final error = 9.566473471908536 %


Méthode de pruning avec variable antithétique

In [18]:
import numpy as np
import scipy.stats as sps
from time import time
import time

r, sigma, X_0, K = 0.1, 0.2, 100, 100

m, b, n = 3, 50, 150
delta_t = 3/m

tic = time.perf_counter()

def Vrecursif(x,y,k):
    if k==m:
        return [(np.maximum(K-x,0)+np.maximum(K-y,0))/2,(np.maximum(K-x,0)+np.maximum(K-y,0))/2]
    else:
        m1=m-k
        d11=(np.log(x/K)+(r+sigma**2/2)*m1)/(sigma*np.sqrt(m1))
        d21=d11-sigma*np.sqrt(m1)
        EspEuro1= (-x*sps.norm.cdf(-d11) + K*np.exp(-r*m1)*sps.norm.cdf(-d21))
        d12=(np.log(y/K)+(r+sigma**2/2)*m1)/(sigma*np.sqrt(m1))
        d22=d12-sigma*np.sqrt(m1)
        EspEuro2= (-y*sps.norm.cdf(-d12) + K*np.exp(-r*m1)*sps.norm.cdf(-d22))
        EspEuro = (EspEuro1 + EspEuro2)/2  #valeur moyenne des options européennes expirant en m partant de x et y.
        if (np.maximum(K-x,0)+np.maximum(K-y,0))/2<EspEuro:
            Ran = np.random.randn()
            xplus=x*np.exp((r-(sigma**2)/2)*delta_t + sigma*np.sqrt(delta_t)*Ran)
            xmoins=y*np.exp((r-(sigma**2)/2)*delta_t - sigma*np.sqrt(delta_t)*Ran)
            return Vrecursif(xplus,xmoins,k+1)
        else:
            Vi=np.zeros((b,2))
            for i in range(b):
                Ran = np.random.randn()
                xplus=x*np.exp((r-(sigma**2)/2)*delta_t + sigma*np.sqrt(delta_t)*Ran)
                xmoins=y*np.exp((r-(sigma**2)/2)*delta_t - sigma*np.sqrt(delta_t)*Ran)
                Vi[i,:]=Vrecursif(xplus,xmoins,k+1)
            VTemp=np.zeros(b)  
            for h in range(b):
                hlist=[i for i in range(b) if i not in [h]]
                V1=np.mean(Vi[hlist,1])
                V2=Vi[h,1]
                a=(np.maximum(K-x,0)+np.maximum(K-y,0))/2
                if V1<=a:
                    VTemp[h]=a
                else:
                    VTemp[h]=V2
            return([np.maximum((np.maximum(K-x,0)+np.maximum(K-y,0))/2,np.mean(Vi[:,0])),np.mean(VTemp)])
    
Sample = np.zeros((n,2))
for k in range(n):
        Sample[k,:] = Vrecursif(X_0,X_0,0)

toc = time.perf_counter()

print("Variable antithétique et méthode de pruning")
print("Time=",toc-tic)
Estimateur=np.mean(Sample[:,0])
Ecart=np.std(Sample[:,0])
print("Vo(n,b)=",Estimateur," Error =",100*2*1.96*Ecart/(np.sqrt(n)*Estimateur),"%")
print("Confidence interval at 95%=",[Estimateur-1.96*Ecart/np.sqrt(n),Estimateur+1.96*Ecart/np.sqrt(n)])
EstimateurLow=np.mean(Sample[:,1])
EcartLow=np.std(Sample[:,1])
print("Low Vo(n,b)=",EstimateurLow," Error =",100*2*1.96*EcartLow/(np.sqrt(n)*EstimateurLow),"%")
print("Low Confidence interval at 95%=",[EstimateurLow-1.96*EcartLow/np.sqrt(n),EstimateurLow+1.96*EcartLow/np.sqrt(n)])
print("Final Confidence interval at 95% =",[EstimateurLow-1.96*EcartLow/np.sqrt(n),Estimateur+1.96*Ecart/np.sqrt(n)])
FE=(EstimateurLow-1.96*EcartLow2/np.sqrt(n) + Estimateur+1.96*Ecart/np.sqrt(n))/2
EE=Estimateur+1.96*Ecart/np.sqrt(n) - (EstimateurLow-1.96*EcartLow/np.sqrt(n))
print("Final estimator =",FE,"Final error =",100*EE/FE,"%")

Variable antithétique et méthode de pruning
Time= 5.144513099999983
Vo(n,b)= 5.613800141871699  Error = 25.581827018303176 %
Confidence interval at 95%= [4.895743821148261, 6.331856462595137]
Low Vo(n,b)= 5.547779606284297  Error = 25.99599006239993 %
Low Confidence interval at 95%= [4.826679488717539, 6.268879723851055]
Final Confidence interval at 95% = [4.826679488717539, 6.331856462595137]
Final estimator = 5.7685694699743415 Final error = 26.09272509782729 %


Variable antithétique et last step removal

In [11]:
import numpy as np
import scipy.stats as sps
from time import time
import time

r, sigma, X_0, K = 0.1, 0.2, 100, 100

m, b, n = 3, 10, 50
delta_t = 3/m

tic = time.perf_counter()

def Vrecursif(x,y,k):
    if k==m-1:
        d11=(np.log(x/K)+(r+sigma**2/2)*1)/(sigma*np.sqrt(1))
        d21=d11-sigma*np.sqrt(1)
        EspEuro1= (-x*sps.norm.cdf(-d11) + K*np.exp(-r*1)*sps.norm.cdf(-d21))
        d12=(np.log(y/K)+(r+sigma**2/2)*1)/(sigma*np.sqrt(1))
        d22=d12-sigma*np.sqrt(1)
        EspEuro2= (-y*sps.norm.cdf(-d12) + K*np.exp(-r*1)*sps.norm.cdf(-d22))
        EspEuro = (EspEuro1 + EspEuro2)/2  #valeur moyenne des options européennes expirant en m partant de x et y.
        return [np.maximum((np.maximum(K-x,0)+np.maximum(K-y,0))/2,EspEuro),np.maximum((np.maximum(K-x,0)+np.maximum(K-y,0))/2,EspEuro)]
    else:
        Vi=np.zeros((b,2))
        for i in range(b):
            Ran = np.random.randn()
            xplus=x*np.exp((r-(sigma**2)/2)*delta_t + sigma*np.sqrt(delta_t)*Ran)
            xmoins=y*np.exp((r-(sigma**2)/2)*delta_t - sigma*np.sqrt(delta_t)*Ran)
            Vi[i,:]=Vrecursif(xplus,xmoins,k+1)
        VTemp=np.zeros(b)  #calcul pour l'estimateur bas
        for h in range(b):
            hlist=[i for i in range(b) if i not in [h]]
            V1=np.exp(-r)*np.mean(Vi[hlist,1])
            V2=np.exp(-r)*Vi[h,1]
            a=(np.maximum(K-x,0)+np.maximum(K-y,0))/2
            if V1<=a:
                VTemp[h]=a
            else:
                VTemp[h]=V2
        return([np.maximum((np.maximum(K-x,0)+np.maximum(K-y,0))/2,np.exp(-r)*np.mean(Vi[:,0])),np.mean(VTemp)])
    
Sample = np.zeros((n,2))
for k in range(n):
        Sample[k,:] = Vrecursif(X_0,X_0,0)

toc = time.perf_counter()

print("Variable antithétique et last step removal")
print("Time=",toc-tic)
Estimateur=np.mean(Sample[:,0])
Ecart=np.std(Sample[:,0])
print("Vo(n,b)=",Estimateur," Error =",100*2*1.96*Ecart/(np.sqrt(n)*Estimateur),"%")
print("Confidence interval at 95%=",[Estimateur-1.96*Ecart/np.sqrt(n),Estimateur+1.96*Ecart/np.sqrt(n)])
EstimateurLow=np.mean(Sample[:,1])
EcartLow=np.std(Sample[:,1])
print("Low Vo(n,b)=",EstimateurLow," Error =",100*2*1.96*EcartLow/(np.sqrt(n)*EstimateurLow),"%")
print("Low Confidence interval at 95%=",[EstimateurLow-1.96*EcartLow/np.sqrt(n),EstimateurLow+1.96*EcartLow/np.sqrt(n)])
print("Final Confidence interval at 95% =",[EstimateurLow-1.96*EcartLow/np.sqrt(n),Estimateur+1.96*Ecart/np.sqrt(n)])
FE=(EstimateurLow-1.96*EcartLow2/np.sqrt(n) + Estimateur+1.96*Ecart/np.sqrt(n))/2
EE=Estimateur+1.96*Ecart/np.sqrt(n) - (EstimateurLow-1.96*EcartLow/np.sqrt(n))
print("Final estimator =",FE,"Final error =",100*EE/FE,"%")

Variable antithétique et last step removal
Time= 1.6130592000000092
Vo(n,b)= 4.9273732490179984  Error = 10.898095169387364 %
Confidence interval at 95%= [4.658878336003541, 5.195868162032456]
Low Vo(n,b)= 4.738460997377503  Error = 11.35026454776813 %
Low Confidence interval at 95%= [4.469547068029923, 5.007374926725083]
Final Confidence interval at 95% = [4.469547068029923, 5.195868162032456]
Final estimator = 4.670553365327715 Final error = 15.55107151530361 %


Variable antithétique, last step removal et pruning 

In [12]:
import numpy as np
import scipy.stats as sps
from time import time
import time

r, sigma, X_0, K = 0.1, 0.2, 100, 100

m, b, n = 3, 50, 700
delta_t = 3/m

tic = time.perf_counter()

def Vrecursif(x,y,k):
    m1=m-k
    d11=(np.log(x/K)+(r+sigma**2/2)*m1)/(sigma*np.sqrt(m1))
    d21=d11-sigma*np.sqrt(m1)
    EspEuro1= (-x*sps.norm.cdf(-d11) + K*np.exp(-r*m1)*sps.norm.cdf(-d21))
    d12=(np.log(y/K)+(r+sigma**2/2)*m1)/(sigma*np.sqrt(m1))
    d22=d12-sigma*np.sqrt(m1)
    EspEuro2= (-y*sps.norm.cdf(-d12) + K*np.exp(-r*m1)*sps.norm.cdf(-d22))
    EspEuro = (EspEuro1 + EspEuro2)/2       #valeur moyenne des options européennes expirant en m partant de x et y.
    if k==m-1: #on prend le max entre la valeur à ce moment et la valeur théorique en m (obtenue via EspEuro)
        return [np.maximum((np.maximum(K-x,0)+np.maximum(K-y,0))/2,EspEuro),np.maximum((np.maximum(K-x,0)+np.maximum(K-y,0))/2,EspEuro)]
    elif (np.maximum(K-x,0)+np.maximum(K-y,0))/2<EspEuro:
        Ran = np.random.randn()
        xplus=x*np.exp((r-(sigma**2)/2)*delta_t + sigma*np.sqrt(delta_t)*Ran)
        xmoins=y*np.exp((r-(sigma**2)/2)*delta_t - sigma*np.sqrt(delta_t)*Ran)
        return [np.exp(-r)*z for z in Vrecursif(xplus,xmoins,k+1)]
    else:
        Vi=np.zeros((b,2))
        for i in range(b):
            Ran = np.random.randn()
            xplus=x*np.exp((r-(sigma**2)/2)*delta_t + sigma*np.sqrt(delta_t)*Ran)
            xmoins=y*np.exp((r-(sigma**2)/2)*delta_t - sigma*np.sqrt(delta_t)*Ran)
            Vi[i,:]=Vrecursif(xplus,xmoins,k+1)
        VTemp=np.zeros(b)  #calcul pour l'estimateur bas
        for h in range(b):
            hlist=[i for i in range(b) if i not in [h]]
            V1=np.exp(-r)*np.mean(Vi[hlist,1])
            V2=np.exp(-r)*Vi[h,1]
            a=(np.maximum(K-x,0)+np.maximum(K-y,0))/2
            if V1<=a:
                VTemp[h]=a
            else:
                VTemp[h]=V2
        return([np.maximum((np.maximum(K-x,0)+np.maximum(K-y,0))/2,np.exp(-r)*np.mean(Vi[:,0])),np.mean(VTemp)])
    
Sample = np.zeros((n,2))
for k in range(n):
        Sample[k,:] = Vrecursif(X_0,X_0,0)

toc = time.perf_counter()

print("Variable antithétique, last step removal et pruning")
print("Time=",toc-tic)
Estimateur=np.mean(Sample[:,0])
Ecart=np.std(Sample[:,0])
print("Vo(n,b)=",Estimateur," Error =",100*2*1.96*Ecart/(np.sqrt(n)*Estimateur),"%")
print("Confidence interval at 95%=",[Estimateur-1.96*Ecart/np.sqrt(n),Estimateur+1.96*Ecart/np.sqrt(n)])
EstimateurLow=np.mean(Sample[:,1])
EcartLow=np.std(Sample[:,1])
print("Low Vo(n,b)=",EstimateurLow," Error =",100*2*1.96*EcartLow/(np.sqrt(n)*EstimateurLow),"%")
print("Low Confidence interval at 95%=",[EstimateurLow-1.96*EcartLow/np.sqrt(n),EstimateurLow+1.96*EcartLow/np.sqrt(n)])
print("Final Confidence interval at 95% =",[EstimateurLow-1.96*EcartLow/np.sqrt(n),Estimateur+1.96*Ecart/np.sqrt(n)])
FE=(EstimateurLow-1.96*EcartLow2/np.sqrt(n) + Estimateur+1.96*Ecart/np.sqrt(n))/2
EE=Estimateur+1.96*Ecart/np.sqrt(n) - (EstimateurLow-1.96*EcartLow/np.sqrt(n))
print("Final estimator =",FE,"Final error =",100*EE/FE,"%")

Variable antithétique, last step removal et pruning
Time= 5.841991699999994
Vo(n,b)= 4.817139807242423  Error = 11.551665833697896 %
Confidence interval at 95%= [4.538909860605082, 5.095369753879765]
Low Vo(n,b)= 4.78540550452811  Error = 11.66136377794208 %
Low Confidence interval at 95%= [4.506383732461766, 5.064427276594453]
Final Confidence interval at 95% = [4.506383732461766, 5.095369753879765]
Final estimator = 4.861114947684317 Final error = 12.116274306547158 %


Variable de contrôle  

In [13]:
import numpy as np
import scipy.stats as sps
from time import time
import time

r, sigma, X_0, K = 0.1, 0.2, 100, 100

m, b, n = 3, 50, 10
delta_t = 3/m

tic = time.perf_counter()

def Vrecursif(x,k):
    if k==m:
        return [np.maximum(K-x,0),np.maximum(K-x,0),np.maximum(K-x,0)]
    else:
        Vi=np.zeros((b,3))
        for i in range(b):
            Vi[i,:]=Vrecursif(x*np.exp((r-(sigma**2)/2)*delta_t + sigma*np.sqrt(delta_t)*np.random.randn()),k+1)
        VTemp=np.zeros(b)  #calcul pour l'estimateur bas
        for h in range(b):
            hlist=[i for i in range(b) if i not in [h]]
            V1=np.exp(-r)*np.mean(Vi[hlist,1])
            V2=np.exp(-r)*Vi[h,1]
            a=np.maximum(K-x,0)
            if V1<=a:
                VTemp[h]=a
            else:
                VTemp[h]=V2
        return([np.maximum(np.maximum(K-x,0),np.exp(-r)*np.mean(Vi[:,0])),np.mean(VTemp),np.exp(-r)*np.mean(Vi[:,2])])
    
Sample0 = np.zeros((n,3))
for k in range(n):
        Sample0[k,:] = Vrecursif(X_0,0)

Varcontrol = Sample0[:,2]    #La variable de contrôle : une option Européenne
d1=(np.log(X_0/K)+(r+sigma**2/2)*m)/(sigma*np.sqrt(m))
d2=d1-sigma*np.sqrt(m)
EspVC= (-X_0*sps.norm.cdf(-d1) + K*np.exp(-r*m)*sps.norm.cdf(-d2))  #L'espérance de la Variable de contrôle
b1=np.cov(Varcontrol,Sample0[:,0])[0,1]/np.var(Varcontrol)
b2=np.cov(Varcontrol,Sample0[:,1])[0,1]/np.var(Varcontrol)

Sample = np.zeros((n,2))
Sample[:,0] = Sample0[:,0] - b1*(Varcontrol-EspVC)
Sample[:,1] = Sample0[:,1] - b2*(Varcontrol-EspVC)
        
toc = time.perf_counter()

print("Variable de contrôle")
print("Time=",toc-tic)
Estimateur=np.mean(Sample[:,0])
Ecart=np.std(Sample[:,0])
print("Vo(n,b)=",Estimateur," Error =",100*2*1.96*Ecart/(np.sqrt(n)*Estimateur),"%")
print("Confidence interval at 95%=",[Estimateur-1.96*Ecart/np.sqrt(n),Estimateur+1.96*Ecart/np.sqrt(n)])
EstimateurLow=np.mean(Sample[:,1])
EcartLow=np.std(Sample[:,1])
print("Low Vo(n,b)=",EstimateurLow," Error =",100*2*1.96*EcartLow/(np.sqrt(n)*EstimateurLow),"%")
print("Low Confidence interval at 95%=",[EstimateurLow-1.96*EcartLow/np.sqrt(n),EstimateurLow+1.96*EcartLow/np.sqrt(n)])
print("Final Confidence interval at 95% =",[EstimateurLow-1.96*EcartLow/np.sqrt(n),Estimateur+1.96*Ecart/np.sqrt(n)])
FE=(EstimateurLow-1.96*EcartLow2/np.sqrt(n) + Estimateur+1.96*Ecart/np.sqrt(n))/2
EE=Estimateur+1.96*Ecart/np.sqrt(n) - (EstimateurLow-1.96*EcartLow/np.sqrt(n))
print("Final estimator =",FE,"Final error =",100*EE/FE,"%")

Variable de contrôle
Time= 43.31617160000002
Vo(n,b)= 4.961605813639987  Error = 3.7576175022896656 %
Confidence interval at 95%= [4.868386729416008, 5.054824897863966]
Low Vo(n,b)= 4.891058481467928  Error = 3.70712155422382 %
Low Confidence interval at 95%= [4.800399739869833, 4.981717223066023]
Final Confidence interval at 95% = [4.800399739869833, 5.054824897863966]
Final estimator = 4.30969885142962 Final error = 5.903548409414625 %


Variable de contrôle avec pruning

In [14]:
import numpy as np
import scipy.stats as sps
from time import time
import time

r, sigma, X_0, K = 0.1, 0.2, 100, 100

m, b, n = 3, 50, 250
delta_t = 3/m

tic = time.perf_counter()

def Vrecursif(x,k):
    if k==m:
        return [np.maximum(K-x,0),np.maximum(K-x,0),np.maximum(K-x,0)]
    else:
        m1=m-k
        d1=(np.log(x/K)+(r+sigma**2/2)*m1)/(sigma*np.sqrt(m1))
        d2=d1-sigma*np.sqrt(m1)
        EspEuro= (-x*sps.norm.cdf(-d1) + K*np.exp(-r*m1)*sps.norm.cdf(-d2))
        if np.maximum(K-x,0)<EspEuro:
            return [np.exp(-r)*z for z in Vrecursif(x*np.exp((r-(sigma**2)/2)*delta_t + sigma*np.sqrt(delta_t)*np.random.randn()),k+1)]
        else:
            Vi=np.zeros((b,3))
            for i in range(b):
                Vi[i,:]=Vrecursif(x*np.exp((r-(sigma**2)/2)*delta_t + sigma*np.sqrt(delta_t)*np.random.randn()),k+1)
            VTemp=np.zeros(b)  #calcul pour l'estimateur bas
            for h in range(b):
                hlist=[i for i in range(b) if i not in [h]]
                V1=np.exp(-r)*np.mean(Vi[hlist,1])
                V2=np.exp(-r)*Vi[h,1]
                a=np.maximum(K-x,0)
                if V1<=a:
                    VTemp[h]=a
                else:
                    VTemp[h]=V2
            return([np.maximum(np.maximum(K-x,0),np.exp(-r)*np.mean(Vi[:,0])),np.mean(VTemp),np.exp(-r)*np.mean(Vi[:,2])])
    
Sample0 = np.zeros((n,3))
for k in range(n):
        Sample0[k,:] = Vrecursif(X_0,0)


Varcontrol = Sample0[:,2]    #La variable de contrôle : une option Européenne
d1=(np.log(X_0/K)+(r+sigma**2/2)*m)/(sigma*np.sqrt(m))
d2=d1-sigma*np.sqrt(m)
EspVC= (-X_0*sps.norm.cdf(-d1) + K*np.exp(-r*m)*sps.norm.cdf(-d2))  #L'espérance de la Variable de contrôle
b1=np.cov(Sample0[:,0],Varcontrol)[0,1]/np.var(Varcontrol)
b2=np.cov(Sample0[:,1],Varcontrol)[0,1]/np.var(Varcontrol)

Sample = np.zeros((n,2))
Sample[:,0] = Sample0[:,0] - b1*(Varcontrol-EspVC)
Sample[:,1] = Sample0[:,1] - b2*(Varcontrol-EspVC)
           
toc = time.perf_counter()

print("Méthode de pruning avec variable de contrôle")
print("Time=",toc-tic)
Estimateur=np.mean(Sample[:,0])
Ecart=np.std(Sample[:,0])
print("Vo(n,b)=",Estimateur," Error =",100*2*1.96*Ecart/(np.sqrt(n)*Estimateur),"%")
print("Confidence interval at 95%=",[Estimateur-1.96*Ecart/np.sqrt(n),Estimateur+1.96*Ecart/np.sqrt(n)])
EstimateurLow=np.mean(Sample[:,1])
EcartLow=np.std(Sample[:,1])
print("Low Vo(n,b)=",EstimateurLow," Error =",100*2*1.96*EcartLow/(np.sqrt(n)*EstimateurLow),"%")
print("Low Confidence interval at 95%=",[EstimateurLow-1.96*EcartLow/np.sqrt(n),EstimateurLow+1.96*EcartLow/np.sqrt(n)])
print("Final Confidence interval at 95% =",[EstimateurLow-1.96*EcartLow/np.sqrt(n),Estimateur+1.96*Ecart/np.sqrt(n)])
FE=(EstimateurLow-1.96*EcartLow2/np.sqrt(n) + Estimateur+1.96*Ecart/np.sqrt(n))/2
EE=Estimateur+1.96*Ecart/np.sqrt(n) - (EstimateurLow-1.96*EcartLow/np.sqrt(n))
print("Final estimator =",FE,"Final error =",100*EE/FE,"%")

Méthode de pruning avec variable de contrôle
Time= 5.131938300000002
Vo(n,b)= 4.965292848962432  Error = 9.897689598824766 %
Confidence interval at 95%= [4.71956821203096, 5.211017485893904]
Low Vo(n,b)= 4.9172880230540175  Error = 10.367158214849962 %
Low Confidence interval at 95%= [4.662396508439079, 5.172179537668956]
Final Confidence interval at 95% = [4.662396508439079, 5.211017485893904]
Final estimator = 4.931504186826696 Final error = 11.124820271274059 %


Variable de contrôle avec variable antithétique et pruning

In [15]:
import numpy as np
import scipy.stats as sps
from time import time
import time

r, sigma, X_0, K = 0.1, 0.2, 100, 100

m, b, n = 3, 50, 100
delta_t = 3/m

tic = time.perf_counter()

def Vrecursif(x,y,k):
    if k==m:
        return [(np.maximum(K-x,0)+np.maximum(K-y,0))/2,(np.maximum(K-x,0)+np.maximum(K-y,0))/2,(np.maximum(K-x,0)+np.maximum(K-y,0))/2]
    else:
        m1=m-k
        d11=(np.log(x/K)+(r+sigma**2/2)*m1)/(sigma*np.sqrt(m1))
        d21=d11-sigma*np.sqrt(m1)
        EspEuro1= (-x*sps.norm.cdf(-d11) + K*np.exp(-r*m1)*sps.norm.cdf(-d21))
        d12=(np.log(y/K)+(r+sigma**2/2)*m1)/(sigma*np.sqrt(m1))
        d22=d12-sigma*np.sqrt(m1)
        EspEuro2= (-y*sps.norm.cdf(-d12) + K*np.exp(-r*m1)*sps.norm.cdf(-d22))
        EspEuro = (EspEuro1 + EspEuro2)/2       #valeur moyenne des options européennes expirant en m partant de x et y. 
        if (np.maximum(K-x,0)+np.maximum(K-y,0))/2<EspEuro:
            Ran = np.random.randn()
            xplus=x*np.exp((r-(sigma**2)/2)*delta_t + sigma*np.sqrt(delta_t)*Ran)
            xmoins=y*np.exp((r-(sigma**2)/2)*delta_t - sigma*np.sqrt(delta_t)*Ran)
            return [np.exp(-r)*z for z in Vrecursif(xplus,xmoins,k+1)]
        else:
            Vi=np.zeros((b,3))
            for i in range(b):
                Ran = np.random.randn()
                xplus=x*np.exp((r-(sigma**2)/2)*delta_t + sigma*np.sqrt(delta_t)*Ran)
                xmoins=y*np.exp((r-(sigma**2)/2)*delta_t - sigma*np.sqrt(delta_t)*Ran)
                Vi[i,:]=Vrecursif(xplus,xmoins,k+1)
            VTemp=np.zeros(b)  #calcul pour l'estimateur bas
            for h in range(b):
                hlist=[i for i in range(b) if i not in [h]]
                V1=np.exp(-r)*np.mean(Vi[hlist,1])
                V2=np.exp(-r)*Vi[h,1]
                a=(np.maximum(K-x,0)+np.maximum(K-y,0))/2
                if V1<=a:
                    VTemp[h]=a
                else:
                    VTemp[h]=V2
            return([np.maximum((np.maximum(K-x,0)+np.maximum(K-y,0))/2,np.exp(-r)*np.mean(Vi[:,0])),np.mean(VTemp),np.exp(-r)*np.mean(Vi[:,2])])
    
Sample0 = np.zeros((n,3))
for k in range(n):
        Sample0[k,:] = Vrecursif(X_0,X_0,0)

Varcontrol = Sample0[:,2]    #La variable de contrôle : une option Européenne
d1=(np.log(X_0/K)+(r+sigma**2/2)*m)/(sigma*np.sqrt(m))
d2=d1-sigma*np.sqrt(m)
EspVC= (-X_0*sps.norm.cdf(-d1) + K*np.exp(-r*m)*sps.norm.cdf(-d2))  #L'espérance de la Variable de contrôle
b1=np.cov(Sample0[:,0],Varcontrol)[0,1]/np.var(Varcontrol)
b2=np.cov(Sample0[:,1],Varcontrol)[0,1]/np.var(Varcontrol)

Sample = np.zeros((n,2))
Sample[:,0] = Sample0[:,0] - b1*(Varcontrol-EspVC)
Sample[:,1] = Sample0[:,1] - b2*(Varcontrol-EspVC)         
        
toc = time.perf_counter()

print("Variable de contrôle, variable antithétique et pruning")
print("Time=",toc-tic)
Estimateur=np.mean(Sample[:,0])
Ecart=np.std(Sample[:,0])
print("Vo(n,b)=",Estimateur," Error =",100*2*1.96*Ecart/(np.sqrt(n)*Estimateur),"%")
print("Confidence interval at 95%=",[Estimateur-1.96*Ecart/np.sqrt(n),Estimateur+1.96*Ecart/np.sqrt(n)])
EstimateurLow=np.mean(Sample[:,1])
EcartLow=np.std(Sample[:,1])
print("Low Vo(n,b)=",EstimateurLow," Error =",100*2*1.96*EcartLow/(np.sqrt(n)*EstimateurLow),"%")
print("Low Confidence interval at 95%=",[EstimateurLow-1.96*EcartLow/np.sqrt(n),EstimateurLow+1.96*EcartLow/np.sqrt(n)])
print("Final Confidence interval at 95% =",[EstimateurLow-1.96*EcartLow/np.sqrt(n),Estimateur+1.96*Ecart/np.sqrt(n)])
FE=(EstimateurLow-1.96*EcartLow2/np.sqrt(n) + Estimateur+1.96*Ecart/np.sqrt(n))/2
EE=Estimateur+1.96*Ecart/np.sqrt(n) - (EstimateurLow-1.96*EcartLow/np.sqrt(n))
print("Final estimator =",FE,"Final error =",100*EE/FE,"%")

Variable de contrôle, variable antithétique et pruning
Time= 4.893279599999971
Vo(n,b)= 5.080994351836642  Error = 11.609950721574513 %
Confidence interval at 95%= [4.786043881629533, 5.375944822043751]
Low Vo(n,b)= 5.029522062253139  Error = 12.198233086429692 %
Low Confidence interval at 95%= [4.722765650109617, 5.336278474396661]
Final Confidence interval at 95% = [4.722765650109617, 5.375944822043751]
Final estimator = 4.992997641086305 Final error = 13.081904276486396 %


Variable de contrôle avec méthode de pruning et last step removal

In [16]:
import numpy as np
import scipy.stats as sps
from time import time
import time

r, sigma, X_0, K = 0.1, 0.2, 100, 100

m, b, n = 3, 50, 5000
delta_t = 3/m

tic = time.perf_counter()

def Vrecursif(x,k):
    m1=m-k
    d1=(np.log(x/K)+(r+sigma**2/2)*m1)/(sigma*np.sqrt(m1))
    d2=d1-sigma*np.sqrt(m1)
    EspEuro= (-x*sps.norm.cdf(-d1) + K*np.exp(-r*m1)*sps.norm.cdf(-d2))
    if k==m-1:
        return [np.maximum(np.maximum(K-x,0),EspEuro),np.maximum(np.maximum(K-x,0),EspEuro),np.exp(-r)*np.maximum(K-x*np.exp((r-(sigma**2)/2)*delta_t + sigma*np.sqrt(delta_t)*np.random.randn()),0)]
    else:
        if np.maximum(K-x,0)<EspEuro:
            return [np.exp(-r)*z for z in Vrecursif(x*np.exp((r-(sigma**2)/2)*delta_t + sigma*np.sqrt(delta_t)*np.random.randn()),k+1)]
        else:
            Vi=np.zeros((b,3))
            for i in range(b):
                Vi[i,:]=Vrecursif(x*np.exp((r-(sigma**2)/2)*delta_t + sigma*np.sqrt(delta_t)*np.random.randn()),k+1)
            VTemp=np.zeros(b)  #calcul pour l'estimateur bas
            for h in range(b):
                hlist=[i for i in range(b) if i not in [h]]
                V1=np.exp(-r)*np.mean(Vi[hlist,1])
                V2=np.exp(-r)*Vi[h,1]
                a=np.maximum(K-x,0)
                if V1<=a:
                    VTemp[h]=a
                else:
                    VTemp[h]=V2
            return([np.maximum(np.maximum(K-x,0),np.exp(-r)*np.mean(Vi[:,0])),np.mean(VTemp),np.exp(-r)*np.mean(Vi[:,2])])
    
Sample0 = np.zeros((n,3))
for k in range(n):
        Sample0[k,:] = Vrecursif(X_0,0)


Varcontrol = Sample0[:,2]    #La variable de contrôle : une option Européenne
d1=(np.log(X_0/K)+(r+sigma**2/2)*m)/(sigma*np.sqrt(m))
d2=d1-sigma*np.sqrt(m)
EspVC= (-X_0*sps.norm.cdf(-d1) + K*np.exp(-r*m)*sps.norm.cdf(-d2))  #L'espérance de la Variable de contrôle
b1=np.cov(Sample0[:,0],Varcontrol)[0,1]/np.var(Varcontrol)
b2=np.cov(Sample0[:,1],Varcontrol)[0,1]/np.var(Varcontrol)

Sample = np.zeros((n,2))
Sample[:,0] = Sample0[:,0] - b1*(Varcontrol-EspVC)
Sample[:,1] = Sample0[:,1] - b2*(Varcontrol-EspVC)
           
toc = time.perf_counter()

print("Variable de contrôle avec méthode de pruning et last step removal")
print("Time=",toc-tic)
Estimateur=np.mean(Sample[:,0])
Ecart=np.std(Sample[:,0])
print("Vo(n,b)=",Estimateur," Error =",100*2*1.96*Ecart/(np.sqrt(n)*Estimateur),"%")
print("Confidence interval at 95%=",[Estimateur-1.96*Ecart/np.sqrt(n),Estimateur+1.96*Ecart/np.sqrt(n)])
EstimateurLow=np.mean(Sample[:,1])
EcartLow=np.std(Sample[:,1])
print("Low Vo(n,b)=",EstimateurLow," Error =",100*2*1.96*EcartLow/(np.sqrt(n)*EstimateurLow),"%")
print("Low Confidence interval at 95%=",[EstimateurLow-1.96*EcartLow/np.sqrt(n),EstimateurLow+1.96*EcartLow/np.sqrt(n)])
print("Final Confidence interval at 95% =",[EstimateurLow-1.96*EcartLow/np.sqrt(n),Estimateur+1.96*Ecart/np.sqrt(n)])
FE=(EstimateurLow-1.96*EcartLow2/np.sqrt(n) + Estimateur+1.96*Ecart/np.sqrt(n))/2
EE=Estimateur+1.96*Ecart/np.sqrt(n) - (EstimateurLow-1.96*EcartLow/np.sqrt(n))
print("Final estimator =",FE,"Final error =",100*EE/FE,"%")

Variable de contrôle avec méthode de pruning et last step removal
Time= 15.115092600000025
Vo(n,b)= 5.107601305818697  Error = 5.11540441678263 %
Confidence interval at 95%= [4.9769640744239485, 5.238238537213446]
Low Vo(n,b)= 5.0762380022212525  Error = 5.158658916398386 %
Low Confidence interval at 95%= [4.945305100061658, 5.207170904380847]
Final Confidence interval at 95% = [4.945305100061658, 5.238238537213446]
Final estimator = 5.127577148279623 Final error = 5.712901604026995 %


Variable de contrôle, variable antithétique, pruning et last step removal

In [17]:
import numpy as np
import scipy.stats as sps
from time import time
import time

r, sigma, X_0, K = 0.1, 0.2, 100, 100

m, b, n = 3, 50, 1500
delta_t = 3/m

tic = time.perf_counter()

def Vrecursif(x,y,k):
    m1=m-k
    d11=(np.log(x/K)+(r+sigma**2/2)*m1)/(sigma*np.sqrt(m1))
    d21=d11-sigma*np.sqrt(m1)
    EspEuro1= (-x*sps.norm.cdf(-d11) + K*np.exp(-r*m1)*sps.norm.cdf(-d21))
    d12=(np.log(y/K)+(r+sigma**2/2)*m1)/(sigma*np.sqrt(m1))
    d22=d12-sigma*np.sqrt(m1)
    EspEuro2= (-y*sps.norm.cdf(-d12) + K*np.exp(-r*m1)*sps.norm.cdf(-d22))
    EspEuro = (EspEuro1 + EspEuro2)/2       #valeur moyenne des options européennes expirant en m partant de x et y.
    if k==m-1:
        Ran = np.random.randn()
        xplus=x*np.exp((r-(sigma**2)/2)*delta_t + sigma*np.sqrt(delta_t)*Ran)
        xmoins=y*np.exp((r-(sigma**2)/2)*delta_t - sigma*np.sqrt(delta_t)*Ran)
        return [np.maximum((np.maximum(K-x,0)+np.maximum(K-y,0))/2,EspEuro),np.maximum((np.maximum(K-x,0)+np.maximum(K-y,0))/2,EspEuro),np.exp(-r)*(np.maximum(K-xplus,0)+np.maximum(K-xmoins,0))/2]
    else: 
        if (np.maximum(K-x,0)+np.maximum(K-y,0))/2<EspEuro:
            Ran = np.random.randn()
            xplus=x*np.exp((r-(sigma**2)/2)*delta_t + sigma*np.sqrt(delta_t)*Ran)
            xmoins=y*np.exp((r-(sigma**2)/2)*delta_t - sigma*np.sqrt(delta_t)*Ran)
            return [np.exp(-r)*z for z in Vrecursif(xplus,xmoins,k+1)]
        else:
            Vi=np.zeros((b,3))
            for i in range(b):
                Ran = np.random.randn()
                xplus=x*np.exp((r-(sigma**2)/2)*delta_t + sigma*np.sqrt(delta_t)*Ran)
                xmoins=y*np.exp((r-(sigma**2)/2)*delta_t - sigma*np.sqrt(delta_t)*Ran)
                Vi[i,:]=Vrecursif(xplus,xmoins,k+1)
            VTemp=np.zeros(b)  #calcul pour l'estimateur bas
            for h in range(b):
                hlist=[i for i in range(b) if i not in [h]]
                V1=np.exp(-r)*np.mean(Vi[hlist,1])
                V2=np.exp(-r)*Vi[h,1]
                a=(np.maximum(K-x,0)+np.maximum(K-y,0))/2
                if V1<=a:
                    VTemp[h]=a
                else:
                    VTemp[h]=V2
            return([np.maximum((np.maximum(K-x,0)+np.maximum(K-y,0))/2,np.exp(-r)*np.mean(Vi[:,0])),np.mean(VTemp),np.exp(-r)*np.mean(Vi[:,2])])
    
Sample0 = np.zeros((n,3))
for k in range(n):
        Sample0[k,:] = Vrecursif(X_0,X_0,0)

Varcontrol = Sample0[:,2]    #La variable de contrôle : une option Européenne
d1=(np.log(X_0/K)+(r+sigma**2/2)*m)/(sigma*np.sqrt(m))
d2=d1-sigma*np.sqrt(m)
EspVC= (-X_0*sps.norm.cdf(-d1) + K*np.exp(-r*m)*sps.norm.cdf(-d2))  #L'espérance de la Variable de contrôle
b1=np.cov(Sample0[:,0],Varcontrol)[0,1]/np.var(Varcontrol)
b2=np.cov(Sample0[:,1],Varcontrol)[0,1]/np.var(Varcontrol)

Sample = np.zeros((n,2))
Sample[:,0] = Sample0[:,0] - b1*(Varcontrol-EspVC)
Sample[:,1] = Sample0[:,1] - b2*(Varcontrol-EspVC)         
        
toc = time.perf_counter()

print("Variable de contrôle, variable antithétique, pruning et last step removal")
print("Time=",toc-tic)
Estimateur=np.mean(Sample[:,0])
Ecart=np.std(Sample[:,0])
print("Vo(n,b)=",Estimateur," Error =",100*2*1.96*Ecart/(np.sqrt(n)*Estimateur),"%")
print("Confidence interval at 95%=",[Estimateur-1.96*Ecart/np.sqrt(n),Estimateur+1.96*Ecart/np.sqrt(n)])
EstimateurLow=np.mean(Sample[:,1])
EcartLow=np.std(Sample[:,1])
print("Low Vo(n,b)=",EstimateurLow," Error =",100*2*1.96*EcartLow/(np.sqrt(n)*EstimateurLow),"%")
print("Low Confidence interval at 95%=",[EstimateurLow-1.96*EcartLow/np.sqrt(n),EstimateurLow+1.96*EcartLow/np.sqrt(n)])
print("Final Confidence interval at 95% =",[EstimateurLow-1.96*EcartLow/np.sqrt(n),Estimateur+1.96*Ecart/np.sqrt(n)])
FE=(EstimateurLow-1.96*EcartLow2/np.sqrt(n) + Estimateur+1.96*Ecart/np.sqrt(n))/2
EE=Estimateur+1.96*Ecart/np.sqrt(n) - (EstimateurLow-1.96*EcartLow/np.sqrt(n))
print("Final estimator =",FE,"Final error =",100*EE/FE,"%")

Variable de contrôle, variable antithétique, pruning et last step removal
Time= 13.12026320000001
Vo(n,b)= 4.948139700798857  Error = 6.294128772613913 %
Confidence interval at 95%= [4.7924185584903, 5.103860843107413]
Low Vo(n,b)= 4.920717178607838  Error = 6.349396424809456 %
Low Confidence interval at 95%= [4.764499258301083, 5.076935098914593]
Final Confidence interval at 95% = [4.764499258301083, 5.103860843107413]
Final estimator = 4.958135459883149 Final error = 6.844540403386409 %


Finite difference method, implicit scheme

In [3]:
import numpy as np
from time import time
import time
r, sigma, X_0, K, m = 0.1, 0.2, 100, 100, 3
marge, t_iter, x_plusiter = 100, 10, 10                             #on considère que u(.,X_0+marge)=0
xsize = x_plusiter*(X_0 + marge - 4) + 1
X_0get=x_plusiter*(X_0 - 4)

tic = time.perf_counter()

xi = np.linspace(4,X_0+marge,xsize)      #les valeurs de x : xsize éléments allant de 0 à K+marge.
delta_x, delta_t = (xi[1]-xi[0]), 1/t_iter
ximoins=np.delete(xi,0)                #pour la ligne du dessous, reliée à u(i-1)
xiplus=np.delete(xi,xsize-1)           #pour la ligne du dessus, reliée à u(i+1)

Error = (delta_x**2 + delta_t)    #de fait l'erreur est majorée par une constante multipliée par cette quantité.

a= -(-1/delta_t - sigma**2*xi**2/delta_x**2-r)
aplus=-((r)*xiplus/(2*delta_x) + sigma**2*xiplus**2/(2*delta_x**2))
amoins=-(-(r)*ximoins/(2*delta_x) + sigma**2*ximoins**2/(2*delta_x**2))

f=np.zeros(xsize)
f[0]=97*(-(r)*4/(2*delta_x) + sigma**2*4**2/(2*delta_x**2))    #conditions de bords : u(.,3)=97, u(.,X_0+marge)=0

A=np.diag(a)
Aplus=np.diag(aplus,k=1)
Amoins=np.diag(amoins,k=-1)
A = delta_t*(A + Aplus + Amoins)

Gx=np.maximum(K-xi,0)

InvA=np.linalg.inv(A)
Bn=np.linalg.matrix_power(InvA,t_iter)
u=Gx
for i in range(m):
    u=np.dot(Bn,u+f)
    u=np.maximum(u,Gx)
    
toc = time.perf_counter()

print("Finite difference scheme")
print("Condition de convergence =",(r-sigma**2/2)*4*delta_x < sigma**4*4**4/4)
print("Time =",toc-tic)
print("confidence interval =",[u[X_0get]-Error,u[X_0get]+Error])
print("Estimator =",u[X_0get],"Error =",2*Error*100,"%")      #xi[X_0get]=100

Finite difference scheme
Condition de convergence = True
Time = 0.8908451000006608
confidence interval = [4.855960597190139, 5.075960597190139]
Estimator = 4.965960597190139 Error = 21.999999999999986 %
